# Data

Read in the three files: clients.csv, loans.csv, payments.csv. These files are related by the following:
1. The clients file is the parent of the loans file. Each client can have multiple distinct loans. The client_id column links the two files
2. The loans file is the child of the clients file and the parent of the payments file. Each loan can have multiple distinct payments associated with it. The loan_id column links the two files.

With the above datasets, answer the following questions. Show the steps taken to produce your final answer.

In [ ]:
# Importing all packages I will need for answering all questions.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import stats # To run ANOVA.
import random                 # To select samples.
from patsy import dmatrices   # To build design matrices.
import statsmodels.api as sm  # Statistical models.
import statistics


Read the three CSV files into dataframes. Check their structure, shape and desribe their statistics. Examine the data.

In [ ]:
# Read the files.
clients = pd.read_csv('clients.csv')
loans = pd.read_csv('loans.csv')
payments = pd.read_csv('payments.csv')

In [ ]:
# Check their structure.
clients.info()
loans.info()
payments.info()

In [ ]:
# Print their shapes.
print(f'clients: {clients.shape}')
print(f'loans: {loans.shape}')
print(f'payments: {payments.shape}')

In [ ]:
clients.head()

In [ ]:
loans.head()

In [ ]:
payments.head()

Convert all columns with a data into datetime.

In [ ]:
clients['joined'] = pd.to_datetime(clients['joined'])
loans['loan_start'] = pd.to_datetime(loans['loan_start'])
loans['loan_end'] = pd.to_datetime(loans['loan_end'])

# Check their structure.
clients.info()
loans.info()
payments.info()

In [ ]:
# Forgot to convert payment_date in payments.
payments['payment_date'] = pd.to_datetime(payments['payment_date'])
payments.info()

# Section 1 Questions

1. Give the 5 client IDs with the highest mean payment amount
2. How many unique loans have been given out to clients who joined prior to 2001?
3. What is the mean number of payments missed by clients with a credit score of less than 700 and who have missed more than 50 payments?

1. Get the 5 client IDs with the highest mean payment amount.

In [ ]:
# loan_ids with highest mean payment amount.
highest_loan_ids = payments[['loan_id', 'payment_amount']]. \
                   groupby('loan_id'). \
                   mean(). \
                   sort_values(ascending=False, by='payment_amount'). \
                   head(5)
# Client ids with top 5 mean payment amount.
pd.merge(highest_loan_ids, loans, on='loan_id')['client_id']

2. Unique loans given out to clients who joined prior to 2001. I am assuming that prior to means joined on or before 31-Dec-2000.

In [ ]:
clients_prior_to_2001 = clients.loc[clients['joined'].dt.year < 2001]['client_id']
pd.merge(clients_prior_to_2001, loans, on='client_id')

3. The mean number of payments missed by clients with a credit score of less than 700 and who have missed more than 50 payments?

In [ ]:
# Clients with credit score < 700.
clients_poor_cs = clients.loc[clients['credit_score'] < 700]['client_id']
# Loan ids of these clients.
loan_ids_poor_cs = pd.merge(clients_poor_cs, loans, on='client_id')[['client_id', 'loan_id']]
# Payment details of these clients.
payments_poor_cs = pd.merge(loan_ids_poor_cs, payments, on='loan_id')
# Get count of missed payments by client.
missed_payments = payments_poor_cs.loc[payments_poor_cs['missed'] == 1].groupby('client_id').count()['missed']
# Get the mean number of missed payments where the missed count > 50. 
ans3 = missed_payments[missed_payments > 50].mean()
print(f'The mean number of payments missed by clients with a credit score of less than 700 and who have missed more than 50 payments = {ans3}')


# Section 2 Questions

Create the following visualizations:
    
1. Create a histogram of the payment amounts. Briefly describe the distribution.
2. Produce a line plot the cumulative sum of the number of clients by year.
3. Produce a scatter plot of the percentage of payments missed in december for each year in the dataset.

1. Histogram of payment amounts.

In [ ]:
ax = payments['payment_amount'].hist(bins=15)
ax.set_xlabel('payment amount')
ax.set_xlabel('# payments')
ignore = ax.set_title('Distribution of payment amounts')

The distribution has a long tail on the right hand side. Most paymenta are in the range 500 to 1500. A significant number of them are less than 2000 with a very few of them above it.

2. Cumulative sum of number of clients per year.

In [ ]:
clients['joined_year'] = clients['joined'].dt.year
ax = clients[['client_id', 'joined_year']]. \
            groupby('joined_year'). \
            count(). \
            cumsum(). \
            reset_index(). \
            plot('joined_year', 'client_id',label='# clients')
ax.set_xlabel('year')
ax.set_ylabel('# clients')
ignore = ax.set_title('Cumulative sum of clients by year')

The maximum on this plot should match the total number of clients. The maximum is 25, so are the number of rows in the data frame <code>clients</code>.

3. Scatter plot of percentage of payments missed in December of every year.

In [ ]:
payments['payment_year'] = payments['payment_date'].dt.year
payments['payment_month'] = payments['payment_date'].dt.month
dec_count = payments.loc[payments['payment_month'] == 12]\
                        [['loan_id', 'payment_year']]. \
                        groupby('payment_year'). \
                        count(). \
                        rename(columns={'loan_id': 'n_loans'}). \
                        reset_index()
dec_missed_count = payments.loc[((payments['payment_month'] == 12) & (payments['missed'] == 1))] \
                               [['loan_id', 'payment_year']]. \
                               groupby('payment_year'). \
                               count(). \
                               rename(columns={'loan_id': 'n_loans_missed'}). \
                               reset_index()

X = pd.merge(dec_count, dec_missed_count, on='payment_year')
X['pct_missed'] = round(X['n_loans_missed']/X['n_loans'] * 100, 2)
ax = X.plot('payment_year', 'pct_missed', label = '% missed')
ax.set_xlabel('year')
ax.set_ylabel('% missed')
ignore = ax.set_title('Percentage of payments missed in December')

# Section 3 - Modelling

Create a model that will predict whether a person does or does not have diabetes. Use the diabetes.csv dataset. The target column in the dataset is "Outcome". Assume no features leak information about the target.

Your solution should include the below. You may use whichever python libraries you wish to complete the task:
1. Feature engineering
2. Model fitting and performance evaluation
3. A function that takes as arguments: a model, train data, test data, and returns the model's predictions on the test data
4. A function that takes a set of predictions and true values and that validates the predictions using appropriate metrics
5. Anything else you feel is necessary for modelling or improving the performance of your model


__This exercise is intended for you to show your proficiency in machine learning, understanding of the various techniques that can be employed to improve the performance of a model, and your ability to implement those techniques. Please, therefore, show your working at all times. You will be judged more for the above than for the performance of the final model your produce.__

## Data munging.

In [ ]:
all_diab = pd.read_csv('test_diabetes.csv', sep=';')

In [ ]:
all_diab.info()

The attributes <code>Insulin</code> and <code>Outcome</code> have a data type <code>object</code>. They need a closer look.

In [ ]:
all_diab['Insulin'].unique()

All values are interpreted as strings because a few of them are put as 'Zero'. We must convert them to '0'. Finally, we must make everything numeric.

In [ ]:
all_diab.loc[all_diab['Insulin'] =='Zero', 'Insulin'] = 0
all_diab['Insulin'] = all_diab['Insulin'].apply(lambda s: float(s))

In [ ]:
all_diab.info()

Let us now look at unique values of <code>Outcome</code>.

In [ ]:
all_diab['Outcome'].unique()

Some values are marked 'N' and 'Y'. We must replace them with '0' and '1' and finally convert all of them to integers.

In [ ]:
all_diab.loc[all_diab['Outcome'] == 'N', 'Outcome'] = '0'
all_diab.loc[all_diab['Outcome'] == 'Y', 'Outcome'] = '1'
all_diab['Outcome'].unique()

In [ ]:
all_diab['Outcome'] = all_diab['Outcome'].apply(lambda s: int(s))

In [ ]:
all_diab.info()

We now have all values as numbers in the data frame. Yet, there are a few NaNs. How many rows have NaNs?

In [ ]:
bad_data = all_diab[all_diab.applymap(np.isnan).any(True)]

print(f'{round(100 * bad_data.shape[0]/all_diab.shape[0], 2)} % of rows have NaNs')

A whopping 35% of observations have NaNs. We cannot just ignore all of it. How are the NaNs distributed across attributes?

In [ ]:
all_diab.shape[0] - all_diab.describe().loc[['count']]

The number of NaNs for each attribute are limited. They are atmost 6.6% (51/768). Therefore, we can impute them with medians.

In [ ]:
all_diab = all_diab.fillna(all_diab.median())
all_diab.describe()

Age has a minimum of $-1$ and a maximum of $200$. Let if find out how many observations have unreasonable ages.

In [ ]:
all_diab.loc[((all_diab['Age'] < 0) | (all_diab['Age'] > 95)), ]

I will replace these abnormal values with median of age.

In [ ]:
all_diab.loc[((all_diab['Age'] < 0) | (all_diab['Age'] > 95)), 'Age'] = statistics.median(all_diab['Age'])

Check if the values look OK.

In [ ]:
all_diab.describe()

I will save the cleaned data for analysis beyond this test.

In [ ]:
all_diab.to_csv('clean_diabetes.csv')

## Preliminary examination of the data

How skewed are the predictors?

In [ ]:
all_diab.describe().loc[['mean', '50%']] # Compare the mean with the median.

Most means are close to their respective medians, except Insulin. The outliers are likely to emerge from here.

In [ ]:
ax = all_diab['Insulin'].hist(cumulative=True, bins = 20, density=1)
ax.set_xlabel('Insulin')
ax.set_ylabel('Cumulative count')
ignore = ax.set_title('Cumulative distribution of Insulin')

95% of subjects have Insulin < 400. I will create a separate data set without them.

In [ ]:
ax = all_diab['Insulin'].hist(bins=20)
ax.set_xlabel('Insulin')
ax.set_ylabel('# cases')
ignore = ax.set_title('Distribution of Insulin')

Insulin values are indeed skewed a lot and we should not expect the mean and the median to be too close.

In [ ]:
D = all_diab.loc[all_diab['Insulin'] < 400]
D.describe()

I am OK to lose approximately 2.35% of data. Even now the gap between mean and median is large, but let is live with it for the moment.

Are there any correlations among the predictors? We will examine using pair plots.

In [ ]:
all_Xs = ['Pregnancies',
          'Glucose',
          'BloodPressure',
          'SkinThickness',
          'Insulin',
          'BMI',
          'DiabetesPedigreeFunction',
          'Age']
smat = pd.plotting.scatter_matrix(all_diab[all_Xs], figsize=(15, 15))

There are no strong correlations among the predictors except for a trend between <code>BMI</code> and <code>SkinThickness</code>. If I have to choose one among them, I would choose <code>BMI</code> because of the short gap between its mean and median. But I will finalize my decision based on t-test.

In [ ]:
Y = 'Outcome'
for x in all_Xs:
  print(all_diab[[x, Y]].groupby(Y).mean().round(2).transpose())
  print(stats.f_oneway(all_diab.loc[all_diab[Y]==0][x], all_diab.loc[all_diab[Y]==1][x]))
  print('-' * 80)

Even ANOVA recommends <code>BMI</code> over <code>SkinThickness</code>. Further, <code>BloodPressure</code> does not seem to have a strong influence on the <code>Outcome</code>.

## Logistic regression

I will first define a few helper functions.

In [ ]:
def trn_tst_split(data, trn_pct):
    """Split the data set into training and test.
    
        data: the data frame to be split.
        trn_pct: % of training data. Must be between 0 and 1.
        
        Returns a list of two data frames - training and test.
    """
    random.seed(12111842)
    N = data.shape[0]
    n_train = int(N * trn_pct)
    trn_indices = set(random.sample(range(N), n_train))
    tst_indices = set(range(N)) - trn_indices
    
    assert len(trn_indices.intersection(tst_indices)) == 0
    assert len(trn_indices) + len(tst_indices) == N
    
    # Convert them to lists.
    trn_indices = [i for i in trn_indices]
    tst_indices = [i for i in tst_indices]
    
    trn_data = data.iloc[trn_indices, :]
    tst_data = data.iloc[tst_indices, :]
    
    assert trn_data.shape[0] + tst_data.shape[0] == data.shape[0]
    
    return [trn_data, tst_data]


def print_cm_results(cm):
    """Print diagnostics from the confusion matrix."""
    recall = cm[0, 0]/(cm[0, 0] + cm[1, 0])
    precision = cm[0, 0]/(cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1]/(cm[1, 0] + cm[1, 1])
    f1_score = 2*recall*precision/(recall + precision)
    accuracy = np.trace(cm)/np.sum(cm)
    
    print(f'% +ves correctly predicted (recall) = {round(recall * 100, 2)}')
    print(f'% +ves detected out of all (precision) = {round(precision * 100, 2)}')
    print(f'% -ves detected out of all (specificity) = {round(specificity * 100, 2)}')
    print(f'f1 score = {round(f1_score, 2)}')
    print(f'accuracy = {round(accuracy * 100, 2)}')

I next list the 'formulae' (similar to the ones used in R) used to define the linear models.

In [ ]:
# First model takes all X's, ignoring the preliminary investigation.
f_v0 = 'Outcome ~ Pregnancies + Glucose + BloodPressure + SkinThickness + Insulin + BMI + DiabetesPedigreeFunction + Age'
# We know that blood pressure does not matter and skin thickness is correlated to BMI. We drop them.
f_v1 = 'Outcome ~ Pregnancies + Glucose + Insulin + BMI + DiabetesPedigreeFunction + Age'

In [ ]:
def get_model_matrices(D, formula):
    return dmatrices(formula, data=D, return_type='dataframe')

In [ ]:
trn_data, tst_data = trn_tst_split(D, 0.75)

In [ ]:
# Training model matrices for model with version 0.
yn_v0, Xn_v0 = get_model_matrices(trn_data, f_v0)
# Training model matrices for model with version 1.
yn_v1, Xn_v1 = get_model_matrices(trn_data, f_v1)

Run the two models and get their confusion matrices on the **training** data.

In [ ]:
model_v0 = sm.Logit(yn_v0, Xn_v0)
results_v0 = model_v0.fit()
cm_v0 = results_v0.pred_table()
print(results_v0.summary())

print()
print('-' * 80)
print()

model_v1 = sm.Logit(yn_v1, Xn_v1)
results_v1 = model_v1.fit()
cm_v1 = results_v1.pred_table()
print(results_v1.summary())

Key observations in model version 0.
1. The coefficients of <code>Insulin</code> and <code>SkinThickness</code> are not statistically significant.
2. The pseudo-R2 for the model is 26%. Not bad for a biological system.

Key observations in model version 1.
1. <code>Insulin</code> and <code>Age</code> must be dropped.
2. The pseudo-R2 has dropped a bit but not too alarmingly.

In [ ]:
# We know that blood pressure does not matter and skin thickness is correlated to BMI. We drop them.
f_v2 = 'Outcome ~ Pregnancies + Glucose + BMI + DiabetesPedigreeFunction'
# Training model matrices for model with version 2.
yn_v2, Xn_v2 = get_model_matrices(trn_data, f_v2)
model_v2 = sm.Logit(yn_v2, Xn_v2)
results_v2 = model_v2.fit()
cm_v2 = results_v2.pred_table()
print(results_v2.summary())

All regression coefficients are significant but pseudo-R2 has dropped by 0.5%. Let us now look at the confusion matrices on the **training** data for all the models.

In [ ]:
# Print all confusion matrices
cms = [cm_v0, cm_v1, cm_v2]
for i, cm in enumerate(cms):
    print(f'Confusion matrix for version {i}:')
    print_cm_results(cm)
    print('-' * 80)

Version 0 seems to be the best on the training data. We will now find out how these model fare on the test data. 

But before we do that, we need a helper function.  

In [ ]:
def prob_to_outcome(y, threshold = 0.5):
    """Converts probability to a class depending on the threshold."""
    if y < threshold:
        return 0
    else:
        return 1
    
def build_cm(yt, yp):
    """Generates confusion matrix from actual and predicted values."""
    yt['predicted'] = [prob_to_outcome(y) for y in yp]
    yt.columns = ['actual', 'predicted']
    yt['actual'] = yt['actual'].apply(lambda f: int(f))
    cm = pd.crosstab(yt['actual'], yt['predicted']).to_numpy()
    
    return cm
    

In [ ]:
# Get the design matrices.
yt_v0, Xt_v0 = get_model_matrices(tst_data, f_v0)
yt_v1, Xt_v1 = get_model_matrices(tst_data, f_v1)
yt_v2, Xt_v2 = get_model_matrices(tst_data, f_v2)

# Get the predictions.
yp_v0 = results_v0.predict(Xt_v0)
yp_v1 = results_v1.predict(Xt_v1)
yp_v2 = results_v2.predict(Xt_v2)

cm_v0 = build_cm(yt_v0, yp_v0)
cm_v1 = build_cm(yt_v1, yp_v1)
cm_v2 = build_cm(yt_v2, yp_v2)

# Print all confusion matrices
cms = [cm_v0, cm_v1, cm_v2]
for i, cm in enumerate(cms):
    print(f'Confusion matrix for version {i}:')
    print_cm_results(cm)
    print('-' * 80)

Version 2 seems to better than the previous ones. We choose it.

Before closing, I will write a function asked in point 3. Our model and the design matrices depend on the formula. Therefore, instead of passing model as the first parameter, we pass the formula, a string. 

In [ ]:
def run_model_on_test_data(formula, train, test, threshold = 0.5):
    """Get predictions of a model on test data.
    
        formula:   a string describing the linear model.
        train:     the training data set.
        test:      the test data set.
        threshold: a cut off applied to the probabilities to decide the class.
    """
    yn, Xn = get_model_matrices(train, formula)
    yt, Xt = get_model_matrices(test, formula)
    
    model = sm.Logit(yn, Xn)
    results = model.fit()
    yp = results.predict(Xt)
    
    return yp

We will test this function on version 2.

In [ ]:
yp2 = run_model_on_test_data(f_v2, trn_data, tst_data)

Check it the results match.

In [ ]:
assert max(abs(yp2 - yp_v2)) == 0.0

Suggested next steps:
1. The specificity of the model is 57%. It means that 43% of the cases classified as diabetic turn out to be not so. I think in this case, it is OK to suspect someone of being diabetic and putting her on medications than otherwise. Do we need to adjust the threshold to decrease specificity and reduce the false positives? (False positive in this case means that the model declared that the person is not diabetic but she turned out to be one.)
2. I have used one split between training and test data. Ideally, one must cross-validate extensively. It is possible to do 30 cross-validations using 'hold-25' strategy. I do not know if this can be done automatically in Python. In R it is just a function call.
3. I would like to see how Naive Bayes performs on this data. Of the four predictors in the version 2 of the model, two appear Gaussian but <code>Pregnancy</code> and <code>DiabetesPedigreeFunction</code> are conspicuously non-Gaussian. I don't know if I can use a Naive Bayesian classifier on such a combination of predictors.
4. It is always worth the efforts to check if a random forest classifier does a better job.